In [ ]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [ ]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

Descarga de empresas en sistema


In [ ]:
llamadaEmpresaAntiguas = "SELECT Id,Empresa__r.Id, Empresa__r.Name, Empresa__r.Holding__c, Empresa__r.Campa_a__c, Empresa__r.Id_Empresa__c, Empresa__r.Empleador__c, Empresa__r.poliza__c, Empresa__r.RUT_Empresa__c, Empresa__r.Corredor__c,Empresa__r.Padre_Empresa__c, Name FROM Programas_por_Empresas__c WHERE Empresa__r.Padre_Empresa__c = false"
vllamadaEmpresa = sf.smart_query(llamadaEmpresaAntiguas)
vllamadaEmpresa.to_excel("F:/Empresas SF/reportes/empresas_Antiguas.xlsx", index=False)
vllamadaEmpresa.head()

vllamadaEmpresa.shape


In [ ]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  SELECT Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  FROM account where ispersonaccount = false  AND Nueva_Estructura__c = False
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

In [ ]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

In [ ]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

In [ ]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

In [ ]:
query2 = "select P_liza__c, Nombre_de_Empresa__c,Identificador__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c "

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]
df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()

fin empresas y comienzo de rescate de pacientes


In [ ]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone,PersonContactId  from Account where ispersonaccount = True 
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
        "PersonContactId",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
        "PersonContactId": "Id Contacto",
    }
)
Pacientes_SF.shape

In [ ]:
def siPrimerCero(y):
    if len(y) <= 7:
        return "NONE"
    if y[0] == "0":
        y = y[1:]
    return y

In [ ]:
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("–", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].apply(lambda y: siPrimerCero(y))
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()

In [ ]:
Pacientes_Nones1 = Pacientes_SF[(Pacientes_SF["Rut"] == "NONE")]
Pacientes_SF = Pacientes_SF[(Pacientes_SF["Rut"] != "NONE")]
Pacientes_Nones1.to_excel("F:/Empresas SF/reportes/NONE1.xlsx", index=False)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


# buscar empresas desde Paciente_Programas\_\_c relaciones de empresas con pacientes

results = sf.query_all(
"""
select Id,Paciente**c, Programas_por_Empresas**c,padre**c from Paciente_Programas**c  
 """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
columns={
"Id": "Id Pacientes_Programas**c",
"Paciente**c": "Id Account",
"Programas_por_Empresas**c": "Programas_por_Empresas**c",
"padre\_\_c": "Nombre Campaña",
}
)
Paciente_Programas_SF.shape


# Union de Pacientes Programas y Programas por empresa

empresas_unidas = pd.merge(
left=Paciente_Programas_SF,
right=Programas_empresas,
how="left",
left_on="Programas_por_Empresas**c",
right_on="Programas_por_Empresas**c",
)
empresas_unidas.shape


Carga de Archivo carga


In [132]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Care Assistance Colaboradores"
Pais = "Chile"
# Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo.head(5)

,rut1,rut2,rutE,apellidos,nombres,FECHA NACIMIENTO,nacimiento,poliza,email,empleador
0,8123516-3,8123516-3,Andrade Moya Francisco Javier,Andrade Moya,Francisco Javier,26-07-1958,1958-07-26,Titular,franciscoandrade@nexoseguros.cl,Titular
1,9666824-4,9666824-4,Aranguiz Calderon Gladys Del Rosario,Aranguiz Calderon,Gladys Del Rosario,30-03-1964,1964-03-30,Titular,gladysaranguiz@nexoseguros.cl,Titular
2,14070097-5,14070097-5,Bafalluy Montory Pauline,Bafalluy Montory,Pauline,09-11-1981,1981-11-09,Titular,paulinebafalluy@nexosrguros.cl,Titular
3,15747879-6,15747879-6,Berenguela Ramos Pablo,Berenguela Ramos,Pablo,16-01-1984,1984-01-16,Titular,pabloberenguela@nexoseguros.cl,Titular
4,11749785-2,11749785-2,Berrios Araya Carola Del Carmen,Berrios Araya,Carola Del Carmen,28-12-1970,1970-12-28,Titular,carolaberrios@nexoseguros.cl,Titular


In [133]:
Pacientes_Archivo.shape

(54, 10)

In [134]:
Pacientes_IDrut = Pacientes_SF[
    [
        "Rut",
        "Id Account Paciente",
        "Id Contacto",
    ]
]

In [135]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("–", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(" ", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()

Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].astype(str)
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.upper()
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.strip()
Pacientes_Archivo.shape

Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].astype(str)
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.upper()
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace("-", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace("–", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace(".", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\2565835100.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\2565835100.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace(".", "")


(54, 11)

In [136]:
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rut2"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("-", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("–", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(" ", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.upper()
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\2296516763.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")


(54, 12)

In [137]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\1579381385.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\1579381385.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [138]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo

# Pacientes_Nones = Pacientes_Archivo[(Pacientes_Archivo["rut"] == "NONE") | (Pacientes_Archivo["rutT"] == "NONE")]
# Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE") | (Pacientes_Archivo["rutT"] != "NONE")]

,rut1,rut2,rutE,apellidos,nombres,FECHA NACIMIENTO,nacimiento,poliza,email,empleador,rut,rutT
0,8123516-3,8123516-3,ANDRADE MOYA FRANCISCO JAVIER,Andrade Moya,Francisco Javier,26-07-1958,1958-07-26,TITULAR,franciscoandrade@nexoseguros.cl,Titular,81235163,81235163
1,9666824-4,9666824-4,ARANGUIZ CALDERON GLADYS DEL ROSARIO,Aranguiz Calderon,Gladys Del Rosario,30-03-1964,1964-03-30,TITULAR,gladysaranguiz@nexoseguros.cl,Titular,96668244,96668244
2,14070097-5,14070097-5,BAFALLUY MONTORY PAULINE,Bafalluy Montory,Pauline,09-11-1981,1981-11-09,TITULAR,paulinebafalluy@nexosrguros.cl,Titular,140700975,140700975
3,15747879-6,15747879-6,BERENGUELA RAMOS PABLO,Berenguela Ramos,Pablo,16-01-1984,1984-01-16,TITULAR,pabloberenguela@nexoseguros.cl,Titular,157478796,157478796
4,11749785-2,11749785-2,BERRIOS ARAYA CAROLA DEL CARMEN,Berrios Araya,Carola Del Carmen,28-12-1970,1970-12-28,TITULAR,carolaberrios@nexoseguros.cl,Titular,117497852,117497852
5,9982585-5,9982585-5,ESCOBAR WENDEROTH LORETO,Escobar Wenderoth,Loreto,06-10-1965,1965-10-06,TITULAR,loretoescobar@nexoseguros.cl,Titular,99825855,99825855
6,9976052-4,9976052-4,GARCIA ALLENDE FANNY,Garcia Allende,Fanny,07-05-1968,1968-05-07,TITULAR,fannygarcia@nexoseguros.cl,Titular,99760524,99760524
7,11678203-0,11678203-0,GARCIA REYES DANIEL EDUARDO,Garcia Reyes,Daniel Eduardo,24-04-1970,1970-04-24,TITULAR,danielgarcia@nexoseguros.cl,Titular,116782030,116782030
8,12061574-2,12061574-2,GOMEZ ARANCIBIA ALEXIS,Gomez Arancibia,Alexis,27-10-1967,1967-10-27,TITULAR,alexisgomez@nexoseguros.cl,Titular,120615742,120615742
9,20083890-4,20083890-4,GONZALEZ BARRIOS FERNANDA JAVIERA,Gonzalez Barrios,Fernanda Javiera,31-01-1997,1997-01-31,TITULAR,fernanda.gonzalez@nexore.cl,Titular,200838904,200838904


In [ ]:
# Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo22.xlsx", index=False)

In [139]:
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE")]
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rutT"] != "NONE")]
# | (Pacientes_Archivo["rutT"] != "NONE")

PAcientes Programas


In [140]:
query_pacientePrograma = (
    "SELECT Paciente__c,Programas_por_Empresas__r.Empresa__c,Paciente__r.IdentificationId__pc,Id from Paciente_Programas__c where padre__c =  '"
    + campaña
    + "'"
)
pacientePrograma = sf.smart_query(query_pacientePrograma)
pacientePrograma.head()

100%|██████████| 1/1 [00:08<00:00,  8.82s/it]


,Paciente__c,Programas_por_Empresas__r.Empresa__c,Paciente__r.IdentificationId__pc,Id
0,0018c00002amPkiAAE,0018c00002cXclPAAS,155224843,a1Y8c00000C6Uc1EAF
1,0018c00002amV6ZAAU,0018c00002cXclPAAS,240423367,a1Y8c00000C6Uc2EAF
2,0018c00002amVabAAE,0018c00002cXclPAAS,170321103,a1Y8c00000C6Uc3EAF
3,0018c00002amVjuAAE,0018c00002cXclPAAS,26197567K,a1Y8c00000C6Uc4EAF
4,0018c00002amQ0yAAE,0018c00002cXclPAAS,184660083,a1Y8c00000C6Uc5EAF


In [141]:
pacientePrograma = pacientePrograma.rename(
    columns={
        "Id": "Id Paciente Programas",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Programas_por_Empresas__r.Empresa__c": "Id Account Empleador",
        "Paciente__c": "Id Account Paciente",
    }
)
pacientePrograma.head()

,Id Account Paciente,Id Account Empleador,Rut,Id Paciente Programas
0,0018c00002amPkiAAE,0018c00002cXclPAAS,155224843,a1Y8c00000C6Uc1EAF
1,0018c00002amV6ZAAU,0018c00002cXclPAAS,240423367,a1Y8c00000C6Uc2EAF
2,0018c00002amVabAAE,0018c00002cXclPAAS,170321103,a1Y8c00000C6Uc3EAF
3,0018c00002amVjuAAE,0018c00002cXclPAAS,26197567K,a1Y8c00000C6Uc4EAF
4,0018c00002amQ0yAAE,0018c00002cXclPAAS,184660083,a1Y8c00000C6Uc5EAF


In [142]:
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("-", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("–", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace(".", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.upper()
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.strip()
pacientePrograma.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\3172767859.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace(".", "")


(115, 4)

In [143]:
PacientesProgramas_PacientesSF = pd.merge(
    left=pacientePrograma,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)

PacientesProgramas_PacientesSF.head()

account contact relation


In [ ]:
query_accountContact = (
    "select Id,ContactId, AccountId, RutContact__c ,Contact.AccountId from AccountContactRelation where Account.Campa_a__c = '"
    + campaña
    + "'"
)
accountContactRelation = sf.smart_query(query_accountContact)
accountContactRelation.head()

100%|██████████| 1/1 [00:03<00:00,  3.67s/it]


,Id,ContactId,AccountId,RutContact__c,Contact.AccountId
0,07k8c00000S8tnbAAB,0038c000035CVIGAA4,0018c00002cXclPAAS,18674344K,0018c00002amLcVAAU
1,07k8c00000S8tncAAB,0038c000035CZQqAAO,0018c00002cXclPAAS,155224843,0018c00002amPkiAAE
2,07k8c00000S8tndAAB,0038c000035CemlAAC,0018c00002cXclPAAS,240423367,0018c00002amV6ZAAU
3,07k8c00000S8tneAAB,0038c000035CfGoAAK,0018c00002cXclPAAS,170321103,0018c00002amVabAAE
4,07k8c00000S8tnfAAB,0038c000035CfQ5AAK,0018c00002cXclPAAS,26197567K,0018c00002amVjuAAE


In [ ]:
accountContactRelation = accountContactRelation.rename(
    columns={
        "Id": "Id AccountContactRelation",
        "ContactId": "Id Contact Paciente",
        "AccountId": "Id Account Empresa",
        "RutContact__c": "Rut",
        "Contact.AccountId": "Id Account Paciente",
    }
)
accountContactRelation.head()

,Id AccountContactRelation,Id Contact Paciente,Id Account Empresa,Rut,Id Account Paciente
0,07k8c00000S8tnbAAB,0038c000035CVIGAA4,0018c00002cXclPAAS,18674344K,0018c00002amLcVAAU
1,07k8c00000S8tncAAB,0038c000035CZQqAAO,0018c00002cXclPAAS,155224843,0018c00002amPkiAAE
2,07k8c00000S8tndAAB,0038c000035CemlAAC,0018c00002cXclPAAS,240423367,0018c00002amV6ZAAU
3,07k8c00000S8tneAAB,0038c000035CfGoAAK,0018c00002cXclPAAS,170321103,0018c00002amVabAAE
4,07k8c00000S8tnfAAB,0038c000035CfQ5AAK,0018c00002cXclPAAS,26197567K,0018c00002amVjuAAE


In [ ]:
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("-", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("–", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace(".", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.upper()
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.strip()
accountContactRelation.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\2268980097.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace(".", "")


(138, 5)

In [ ]:
AccountContact_PacientesSF = pd.merge(
    left=accountContactRelation,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
AccountContact_PacientesSF.head()

,Id AccountContactRelation,Id Contact Paciente,Id Account Empresa,Rut_x,Id Account Paciente,Rut_y,Id Contacto
0,07k8c00000S8tnbAAB,0038c000035CVIGAA4,0018c00002cXclPAAS,18674344K,0018c00002amLcVAAU,18674344K,0038c000035CVIGAA4
1,07k8c00000S8tncAAB,0038c000035CZQqAAO,0018c00002cXclPAAS,155224843,0018c00002amPkiAAE,155224843,0038c000035CZQqAAO
2,07k8c00000S8tndAAB,0038c000035CemlAAC,0018c00002cXclPAAS,240423367,0018c00002amV6ZAAU,240423367,0038c000035CemlAAC
3,07k8c00000S8tneAAB,0038c000035CfGoAAK,0018c00002cXclPAAS,170321103,0018c00002amVabAAE,170321103,0038c000035CfGoAAK
4,07k8c00000S8tnfAAB,0038c000035CfQ5AAK,0018c00002cXclPAAS,26197567K,0018c00002amVjuAAE,26197567K,0038c000035CfQ5AAK


empresa contacto convenio


In [ ]:
query_empresacontactoconvenio = (
    "select Id,Paciente__r.IdentificationId__pc,  Empresa__c, Paciente__c,N_Documento_Titular__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '"
    + campaña
    + "'"
)
empresacontactoconvenio = sf.smart_query(query_empresacontactoconvenio)
empresacontactoconvenio.head()

100%|██████████| 1/1 [00:02<00:00,  2.24s/it]


,Id,Paciente__r.IdentificationId__pc,Empresa__c,Paciente__c,N_Documento_Titular__c
0,a40RN000000wu6fYAA,202201466,a3yRN0000006reXYAQ,001RN000005OgWJYA0,202201466
1,a40RN000000zMd7YAE,18295294K,a3yRN0000006reXYAQ,001RN000005DRHHYA4,192909147
2,a40RN0000010GozYAE,191710940,a3yRN0000006reXYAQ,0018c00002amLcCAAU,191710940
3,a40RN0000010GqbYAE,182470082,a3yRN0000006reXYAQ,0018c00002amSetAAE,182470082
4,a40RN0000011BorYAE,155455012,a3yRN0000006reXYAQ,001RN000004vRE5YAM,167148344


In [ ]:
empresacontactoconvenio = empresacontactoconvenio.rename(
    columns={
        "Id": "Id EmpresaContactoConvenio",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Empresa__c": "Id Convenio Empresa",
        "Paciente__c": "Id Account Paciente",
        "N_Documento_Titular__c": "Rut Titular",
    }
)
empresacontactoconvenio.head()

,Id EmpresaContactoConvenio,Rut,Id Convenio Empresa,Id Account Paciente,Rut Titular
0,a40RN000000wu6fYAA,202201466,a3yRN0000006reXYAQ,001RN000005OgWJYA0,202201466
1,a40RN000000zMd7YAE,18295294K,a3yRN0000006reXYAQ,001RN000005DRHHYA4,192909147
2,a40RN0000010GozYAE,191710940,a3yRN0000006reXYAQ,0018c00002amLcCAAU,191710940
3,a40RN0000010GqbYAE,182470082,a3yRN0000006reXYAQ,0018c00002amSetAAE,182470082
4,a40RN0000011BorYAE,155455012,a3yRN0000006reXYAQ,001RN000004vRE5YAM,167148344


In [ ]:
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("-", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("–", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(" ", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.upper()
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.strip()
empresacontactoconvenio.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\2240900084.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")


(6, 5)

In [ ]:
empresacontactoconvenio_PacientesSF = pd.merge(
    left=empresacontactoconvenio,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
empresacontactoconvenio_PacientesSF.head()

,Id EmpresaContactoConvenio,Rut_x,Id Convenio Empresa,Id Account Paciente,Rut Titular,Rut_y,Id Contacto
0,a40RN000000wu6fYAA,202201466,a3yRN0000006reXYAQ,001RN000005OgWJYA0,202201466,202201466,003RN00000I0uZTYAZ
1,a40RN000000zMd7YAE,18295294K,a3yRN0000006reXYAQ,001RN000005DRHHYA4,192909147,18295294K,003RN00000H0WauYAF
2,a40RN0000010GozYAE,191710940,a3yRN0000006reXYAQ,0018c00002amLcCAAU,191710940,191710940,0038c000035CVHxAAO
3,a40RN0000010GqbYAE,182470082,a3yRN0000006reXYAQ,0018c00002amSetAAE,182470082,182470082,0038c000035CcL3AAK
4,a40RN0000011BorYAE,155455012,a3yRN0000006reXYAQ,001RN000004vRE5YAM,167148344,155455012,003RN00000FcDGXYA3


In [ ]:
""" if(campaña!="MetLife"):
    pacientePrograma.to_excel("F:/Empresas SF/reportes/paciente_programas.xlsx", index=False) """

Rescatamos todos los ruts Id con su empresa desde la campaña


In [ ]:
Pacientes_Archivo.head()

,rut2,rut1,Nombre,Apellido Paterno,Apellido Materno,apellidos,empleador,poliza,email,celular,...,APELLIDOS.2,RUT.2,NOMBRE.2,APELLIDOS.3,RUT.3,NOMBRE.3,APELLIDOS.4,RUT.4,rut,rutT
0,17919661-1,17919661-1,ALEXANDRA,RODRIGUEZ,NORAMBUENA,RODRIGUEZ NORAMBUENA,NUTRICIONISTA,OPERACIONES,Alexandrarodriguez.nutri@gmail.com,958597502,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,179196611,179196611
1,19165611-3,19165611-3,ALMENDRA ESTEFANI,VILLARROEL,SOTO,VILLARROEL SOTO,ENCARGADA DE GRUPO DE ENFERMERIA,OPERACIONES,Almendra.villarroel@gmail.com,965020998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,191656113,191656113
2,17702217-9,17702217-9,ANDREA IGNACIA,LANDAETA,BUSTAMANTE,LANDAETA BUSTAMANTE,PSICOLOGO ORIENTACION CLINICA,OPERACIONES,Andrealandaeta.b@gmail.com,956272800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,177022179,177022179
3,19171094-0,19171094-0,CARLOS ARMANDO,VILLANUEVA,URTUBIA,VILLANUEVA URTUBIA,SOPORTE INFORMATICA,TECNOLOGIA DE INFORMACION,carlos.armando.villanueva@outlook.com,956639972,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,191710940,191710940
4,16714834-4,16714834-4,CAROLINA PAZ,JARA,GONZÁLEZ,JARA GONZÁLEZ,"COORDINACION DE EQUIPO ORIENTA, FIDELIZACION Y...",OPERACIONES,Carolina.jara.p.c@gmail.com,945506279,...,GUTIÉRREZ JARA,27969477-5,ANDRÉS ENRIQUE,GUTIÉRREZ GALLARDO,15545501-2,NaN,NaN,NaN,167148344,167148344


Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)


Pacientes_Archivo.drop_duplicates(
[
"rut",
"rutT",
"LLaveEmpresa",
"LLAveRut",
],
keep="last",
)
Pacientes_Archivo.shape


In [ ]:
print(campaña)

Care Assistance Colaboradores


In [ ]:
Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

In [ ]:
# Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo.xlsx", index=False)

In [ ]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
# Pacientes_con_id.to_excel("F:/Empresas SF/reportes/Pacientes_con_id.xlsx", index=False)
Pacientes_con_id.shape

(93, 38)

In [ ]:
Pacientes_con_id.head()

,rut2,rut1,Nombre_x,Apellido Paterno,Apellido Materno,apellidos,empleador,poliza,email,celular,...,Id Account Paciente,Rut,Nombre_y,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono,Id Contacto
0,17919661-1,17919661-1,ALEXANDRA,RODRIGUEZ,NORAMBUENA,RODRIGUEZ NORAMBUENA,NUTRICIONISTA,OPERACIONES,Alexandrarodriguez.nutri@gmail.com,958597502,...,0018c00002eTejvAAC,179196611,Alexandra Rodriguez Norambuena,RUT,None,None,None,Mujer,None,0038c000038NfA8AAK
1,19165611-3,19165611-3,ALMENDRA ESTEFANI,VILLARROEL,SOTO,VILLARROEL SOTO,ENCARGADA DE GRUPO DE ENFERMERIA,OPERACIONES,Almendra.villarroel@gmail.com,965020998,...,001RN00000197GHYAY,191656113,Almendra Estefani Villarroel Soto,RUT,None,None,None,None,965020998,003RN000002k5zhYAA
2,17702217-9,17702217-9,ANDREA IGNACIA,LANDAETA,BUSTAMANTE,LANDAETA BUSTAMANTE,PSICOLOGO ORIENTACION CLINICA,OPERACIONES,Andrealandaeta.b@gmail.com,956272800,...,001RN00000197GEYAY,177022179,Andrea Ignacia Landaeta Bustamante,RUT,None,None,None,None,956272800,003RN000002k5zeYAA
3,19171094-0,19171094-0,CARLOS ARMANDO,VILLANUEVA,URTUBIA,VILLANUEVA URTUBIA,SOPORTE INFORMATICA,TECNOLOGIA DE INFORMACION,carlos.armando.villanueva@outlook.com,956639972,...,0018c00002amLcCAAU,191710940,Carlos Villanueva Urtubia,RUT,None,None,None,Hombre,+56956639972,0038c000035CVHxAAO
4,16714834-4,16714834-4,CAROLINA PAZ,JARA,GONZÁLEZ,JARA GONZÁLEZ,"COORDINACION DE EQUIPO ORIENTA, FIDELIZACION Y...",OPERACIONES,Carolina.jara.p.c@gmail.com,945506279,...,0018c00002mUWHtAAO,167148344,Carolina Jara,RUT,None,None,None,None,945506279,0038c00003IeIxoAAF


In [ ]:
# Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)

In [ ]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")


def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""


def fechaValida(fecha):
    fecha = str(fecha)
    fecha1 = str(fecha)
    
    #datetime.strptime(fecha, '%Y-%m-%d')
    
    try:
        fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        date_string = fecha1
        print("date_string =", date_string)
        print("type of date_string =", type(date_string))

        date_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")

        print("date_object =", date_object)
        print("type of date_object =", type(date_object))
        return fecha
    except ValueError:
        return ""
    

In [ ]:
Pacientes_con_id["dertyuiop"] = Pacientes_con_id["nacimiento"]
Pacientes_con_id["derty"] = Pacientes_con_id["nacimiento"].apply(lambda y: fechaValida(y))
Pacientes_con_id.head()

In [ ]:


# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# Subir_pacientes["Email_Seteado__c"] = ""
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )


if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = Pacientes_con_id["email"].astype(str).str.lower()
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(
        lambda y: isValid(y)
    )
    Subir_pacientes["Email_Seteado__c"] = np.where(
        Subir_pacientes["PersonEmail"] != "", "True", "False"
    )

    # Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
#if ("nacimiento" in Pacientes_con_id.columns) == True:    
#    Subir_pacientes["PersonBirthdate"] = pd.to_datetime(Pacientes_con_id["nacimiento"].astype(str), format="%Y-%m-%d")  
if ("nacimiento" in Pacientes_con_id.columns) == True:    
    #try:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"].apply(lambda y: fechaValida(y))
    #except ValueError:
        #Subir_pacientes["PersonBirthdate"] = pd.to_datetime("1900-01-01", format="%Y-%m-%d")
 
if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "H": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "Prefiero no responder",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "M": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon": "Hombre",
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace(".", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("null", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NULL", "")


Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"

Subir_pacientes_Que_no_Estan = Subir_pacientes
Subir_pacientes_Estan = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == True]
Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel(
    "F:/Empresas SF/reportes/Pacientes_Que_No_Estaban.xlsx", index=False
)
Subir_pacientes.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\1533575850.py:93: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\1533575850.py:100: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")


,RutEstaoNo,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
45,False,GUZMÁN ROJAS,197095326,ps.raulguzman@gmail.com,True,99945278,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
46,False,CARMONA CORTÉS,202231837,camila.carmona.cortes@gmail.com,True,991323888,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
47,False,FARFAN MARCELO,27530753K,kiarafarfan21@gmail.com,True,937153949,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
48,False,ROJAS PRENAFETA,201077265,javier.rojaspre@gmail.com,True,99822239,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
49,False,SEPÚLVEDA DÍAZ,197425563,alexandrarodriguez.nutri@gmail.com,True,95859752,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [ ]:
Subir_pacientes.shape

(29, 11)

In [ ]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc)"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return "None"


def traerContact(ide):
    ide = str(ide)
    if len(ide) >= 12:
        idcontact = sf.Account.get(ide)
        return idcontact["PersonContactId"]
    else:
        return "NONES"


def agregarPaciente(listado):
    vListado = listado.split(",")
    
    derty = None
    
        
        
    if (vListado[9] == ""):                               
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],  
                                          
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    else:
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],
                        "PersonBirthdate": vListado[9],
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    # Verify that the Contact was created successfully
    # created_contact = sf.Contact.get(sf_res['id'])
    # print(created_contact)


""" sf_res =None
try:
    # Create the new Contact record in Salesforce
    sf_res = sf.Contact.create(new_contact)
except SalesforceMalformedRequest as e:
    if 'duplicateResut' in e[0]:
        print(f"Malformed request error: {e}")
        # Handle the error here as needed
        #Bypass SF Email duplicate rule by removing the email to create a SF Lead
        del new_contact['Email']
        sf_res = sf.Contact.create(new_contact)


if sf_res is not None:
    # Verify that the Contact was created successfully
    created_contact = sf.Contact.get(sf_res['id'])
    print(created_contact) """

' sf_res =None\ntry:\n    # Create the new Contact record in Salesforce\n    sf_res = sf.Contact.create(new_contact)\nexcept SalesforceMalformedRequest as e:\n    if \'duplicateResut\' in e[0]:\n        print(f"Malformed request error: {e}")\n        # Handle the error here as needed\n        #Bypass SF Email duplicate rule by removing the email to create a SF Lead\n        del new_contact[\'Email\']\n        sf_res = sf.Contact.create(new_contact)\n\n\nif sf_res is not None:\n    # Verify that the Contact was created successfully\n    created_contact = sf.Contact.get(sf_res[\'id\'])\n    print(created_contact) '

In [ ]:
Subir_pacientes.head()

,RutEstaoNo,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
45,False,GUZMÁN ROJAS,197095326,ps.raulguzman@gmail.com,True,99945278,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
46,False,CARMONA CORTÉS,202231837,camila.carmona.cortes@gmail.com,True,991323888,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
47,False,FARFAN MARCELO,27530753K,kiarafarfan21@gmail.com,True,937153949,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
48,False,ROJAS PRENAFETA,201077265,javier.rojaspre@gmail.com,True,99822239,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
49,False,SEPÚLVEDA DÍAZ,197425563,alexandrarodriguez.nutri@gmail.com,True,95859752,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [ ]:
Subir_pacientes.drop_duplicates(
    [
        "IdentificationId__pc",
    ],
    keep="last",
)

Subir_pacientes["Id"] = Subir_pacientes["IdentificationId__pc"].apply(
    lambda y: traerId(y)
)

Subir_pacientes.shape

vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\2950418267.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Subir_pacientes["Id"] = Subir_pacientes["IdentificationId__pc"].apply(


(29, 12)

In [ ]:
Subir_pacientes["Id"] = Subir_pacientes["Id"].astype(str)
Subir_pacientes = Subir_pacientes[Subir_pacientes["Id"].str.len() <= 12]
# Subir_pacientes["numero"] = Subir_pacientes["Id"].str.len()
Subir_pacientes.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\2892125083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Subir_pacientes["Id"] = Subir_pacientes["Id"].astype(str)


(29, 12)

In [ ]:
#Subir_pacientes["dato_account"].head()

In [ ]:
Subir_pacientes.to_excel("F:/Empresas SF/reportes/Todesffff.xlsx", index=False)

In [ ]:
Subir_pacientes["dato_account"] = ""
if ("FirstName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes[
        "FirstName"
    ].astype(str)
if ("LastName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["LastName"].astype(str)
    )
if ("IdentificationId__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["IdentificationId__pc"].astype(str)
    )
if ("RecordTypeId" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["RecordTypeId"].astype(str)
    )
if ("BillingCountry" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCountry"].astype(str)
    )
if ("Identification_Type__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Identification_Type__c"].astype(str)
    )
if ("Pa_s_del_Paciente__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Pa_s_del_Paciente__c"].astype(str)
    )
if ("Cuenta_validada__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Cuenta_validada__c"].astype(str)
    )

if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["PersonEmail"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

if ("PersonBirthdate" in Subir_pacientes.columns) == True:
    #if Subir_pacientes["dato_account"] != "":
        #fecha = pd.to_datetime(Subir_pacientes["PersonBirthdate"].astype(str) + " 00:00:00").isoformat() + "Z" 
        #fecha = pd.to_datetime(Subir_pacientes["PersonBirthdate"].astype(str) + " 00:00:00")                       
        Subir_pacientes["dato_account"] = (Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonBirthdate"].astype(str)
            
        )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("HealthCloudGA__Gender__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["HealthCloudGA__Gender__pc"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("Phone" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["Phone"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingCity" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCity"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingState" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingState"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingStreet" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingStreet"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Email_Seteado__c"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

# Subir_pacientes["dato_account"] =  Subir_pacientes["dato_account"] =

Subir_pacientes["Id_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
)
""" Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
) """

' Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(\n    lambda x: agregarPaciente(x)\n) '

In [ ]:
query2 = "select P_liza__c, Nombre_de_Empresa__c,Identificador__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c WHERE Empresa_Padre__r.Name = '" + campaña + "'"

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]
df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_asociadas.xlsx", index=False)
df2.head()

100%|██████████| 1/1 [00:07<00:00,  7.84s/it]


,P_liza__c,Nombre_de_Empresa__c,Identificador__c,Rut_Empresa__c,Empresa_Padre__r.Name,Holding__c,Plan__c,Pais__c,Activo__c,Id,LLave,dato
0,-,Colaboradores Care Assistance Chile SPA,Colaboradores Care Assistance Chile SPA,-,Care Assistance Colaboradores,-,Plan Gold,Chile,True,a3yRN0000006reXYAQ,-,1


In [131]:
Pacientes_con_id_no = Pacientes_con_id[Pacientes_con_id["RutEstaoNo"] == False]
Pacientes_con_id_no["Id Account Paciente"] = Pacientes_con_id_no["rut"].apply(
    lambda y: traerId(y)
)
Pacientes_con_id_no["Id Contacto"] = Pacientes_con_id_no["Id Account Paciente"].apply(
    lambda y: traerContact(y)
)
Pacientes_con_id_no.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_no.xlsx", index=False
)

Pacientes_con_id_si = Pacientes_con_id[Pacientes_con_id["RutEstaoNo"] == True]
Pacientes_con_id_si.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_si.xlsx", index=False
)

Pacientes_con_id_final = pd.concat(
    [
        Pacientes_con_id_si,
        Pacientes_con_id_no,
    ],
    axis=0,
)

Pacientes_con_id_final.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_final.xlsx", index=False
)

if campaña == "Southbridge Plan Mujer":
    VpacientePrograma = "a1WRN0000007YwD2AU"
    VempresaaAccount = "001RN000003rwVaYAI"
    VempresaConvenio = "a3yRN0000006sVTYAY"
elif campaña == "Southbridge":
    VpacientePrograma = "a1W8c0000083pM9EAI"
    VempresaaAccount = "0018c00002f3Ev2AAE"
    VempresaConvenio = "a3yRN0000006sqYYAQ"
elif campaña == "Construmart":
    VpacientePrograma = "a1W8c0000083q9TEAQ"
    VempresaaAccount = "0018c00002ghJgBAAU"
    VempresaConvenio = "a3yRN0000006rlZYAQ"
elif campaña == "EY":
    VpacientePrograma = "a1W8c0000083laZEAQ"
    VempresaaAccount = "0018c00002eTboVAAS"
    VempresaConvenio = "a3yRN0000006rsTYAQ"
elif campaña == "Colmena":
    VpacientePrograma = "a1W8c0000083j8rEAA"
    VempresaaAccount = "0018c00002cXclQAAS"
    VempresaConvenio = "a3yRN0000006rf5YAA"
elif campaña == "Pedro De Valdivia":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006sVzYAI"
elif campaña == "Forum":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006s3SYAQ"
elif campaña == "Cruz Verde":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006rsqYAA"
elif campaña == "Milan Fajanovic":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006sR4YAI"
elif campaña == "ProChile":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006rpsYAA"
elif campaña == "Farmacias Ahumada":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006rspYAA"
elif campaña == "Minera El Abra":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006snOYAQ"
elif campaña == "Instruvalve":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006s9DYAQ"
elif campaña == "VidaCel":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006suzYAA"
elif campaña == "Care Assistance Colaboradores":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006reXYA"
elif campaña == "Nexo":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006sORYAY"
else:
    VpacientePrograma = "Buscarv"
    VempresaaAccount = "Buscarv"
    VempresaConvenio = "Buscarv"


Pacientes_con_id_final["esonoes"] = Pacientes_con_id_final["rut"].isin(
    PacientesProgramas_PacientesSF["Rut_y"]
)
Subir_programas_empresa = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes"] == False
]
Subir_programas_empresa["Paciente__c"] = Subir_programas_empresa["Id Account Paciente"]
Subir_programas_empresa["Programas_por_Empresas__c"] = VpacientePrograma
Subir_programas_empresa = Subir_programas_empresa[
    [
        # "empresa",
        "poliza",
        "Paciente__c",
        "Programas_por_Empresas__c",
    ]
]
Subir_programas_empresa.to_excel(
    "F:/Empresas SF/reportes/programas_empresa.xlsx", index=False
)


Pacientes_con_id_final["esonoes1"] = Pacientes_con_id_final["rut"].isin(
    AccountContact_PacientesSF["Rut_y"]
)
Subir_AccountContact = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes1"] == False
]
Subir_AccountContact["ContactId"] = Subir_AccountContact["Id Contacto"]
Subir_AccountContact["AccountId"] = VempresaaAccount

Subir_AccountContact = Subir_AccountContact[
    [
        # "empresa",
        "poliza",
        "ContactId",
        "AccountId",
    ]
]
Subir_AccountContact.to_excel(
    "F:/Empresas SF/reportes/account_Contact.xlsx", index=False
)

empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
Pacientes_con_id_final["llaveA"] = Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"]

Pacientes_con_id_final["esonoes2"] = Pacientes_con_id_final["llaveA"].isin(
    empresacontactoconvenio_PacientesSF["llave"]
)
Subir_empresacontactoconvenio = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes2"] == False
]
Subir_empresacontactoconvenio["Paciente__c"] = Subir_empresacontactoconvenio[
    "Id Account Paciente"
]
Subir_empresacontactoconvenio["N_Documento_Titular__c"] = Subir_empresacontactoconvenio[
    "rutT"
]
Subir_empresacontactoconvenio["Empresa__c"] = VempresaConvenio

Subir_empresacontactoconvenio = Subir_empresacontactoconvenio[
    [
        "empleador",
        "rutE",
        "poliza",
        "Paciente__c",
        "N_Documento_Titular__c",
        "Empresa__c",
    ]
]
Subir_empresacontactoconvenio.to_excel(
    "F:/Empresas SF/reportes/empresacontactoconvenio.xlsx", index=False)
Pacientes_con_id_final.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_final2.xlsx", index=False
)


001RN0000060x4fYAA 197095326
001RN0000060x6HYAQ 202231837
001RN0000060wztYAA 27530753K
001RN0000060t4gYAA 201077265
001RN0000060qMbYAI 197425563
001RN0000060x7tYAA 171098092
001RN0000060sbJYAQ 25551056K
001RN0000060x9VYAQ 250777124
001RN0000060wjiYAA 88429559
001RN0000060mxIYAQ 232620588
001RN0000060srXYAQ 76543070
001RN0000060oUSYAY 164716449
001RN0000060uWdYAI 109183822
001RN0000060n0YYAQ 198922889
001RN0000060lQCYAY 252612483
001RN0000060nIEYAY 177838446
001RN0000060rSLYAY 10725126K
001RN0000060oChYAI 10808331K
001RN0000060K1pYAE 142334836
001RN0000060xB7YAI 103851300
001RN0000060xCjYAI 62402822
001RN0000060szUYAQ 272088543
001RN0000060tsJYAQ 109618586
001RN0000060nADYAY 179284057
001RN0000060ld5YAA 235401126
001RN0000060vVzYAI 141277065
001RN0000060vCYYAY 100211610
001RN0000060uI7YAI 208087800
001RN0000060ni6YAA 48803679


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\2072202403.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pacientes_con_id_no["Id Account Paciente"] = Pacientes_con_id_no["rut"].apply(
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\2072202403.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pacientes_con_id_no["Id Contacto"] = Pacientes_con_id_no["Id Account Paciente"].apply(
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2104\2072202403.py:97: SettingWithCopyWarning: 
A value is trying 

In [ ]:
empresacontactoconvenio_PacientesSF.to_excel(
    "F:/Empresas SF/reportes/SFpacientes.xlsx", index=False)

In [ ]:
contactd = sf.Account.get("0018c00002lUyo0AAC")
contactd["PersonContactId"]

In [ ]:
# contactd["PersonContactId"]
print(datetime.now())

In [ ]:
Subir_pacientes.to_excel("F:/Empresas SF/reportes/todes.xlsx", index=False)

query2 = "select P_liza**c, Nombre_de_Empresa**c, Rut_Empresa**c, Empresa_Padre**r.Name, Empresa**r.name,Holding**c, Plan**c, Pais**c, Activo**c, Id from Datos_Empresa**c "

df2 = sf.smart_query(query2)
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()


Subir_pacientes.to_excel("F:/Empresas SF/reportes/111account.xlsx", index=False)


derty = Subir_pacientes["IdentificationId__pc"]
sf.Contact.create({"IdentificationId\_\_pc": "derty", "Email": "example@example.com"})


Subir_pacientes.head()


Subir_pacientes.head()
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account\_\_l.xlsx", index=False)


In [ ]:
""" if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )
 """

In [ ]:
""" if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    ) """

In [ ]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [ ]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)

In [ ]:
campaña_en_pacientes2["rut"].shape

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
rutes = pd.DataFrame(columns=["rut"])
rutes["rut"] = Pacientes_Archivo["rut"]
rutes["rutT"] = Pacientes_Archivo["rut2"]
rutes["Contratante"] = Pacientes_Archivo["Contratante"]
# rutes["poliza"] = Pacientes_Archivo["poliza"]
rutes.head()

In [ ]:
rutes["rut"] = rutes["rut"].astype(str)
rutes["rut"] = rutes["rut"].str.replace(".", "")
rutes["rut"] = rutes["rut"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rut"] = rutes["rut"].str.upper()
rutes["rut"] = rutes["rut"].str.strip()

rutes["rutT"] = rutes["rutT"].astype(str)
rutes["rutT"] = rutes["rutT"].str.replace(".", "")
rutes["rutT"] = rutes["rutT"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rutT"] = rutes["rutT"].str.upper()
rutes["rutT"] = rutes["rutT"].str.strip()

""" rutes["poliza"] = rutes["poliza"].astype(str)
rutes["poliza"] = rutes["poliza"].str.replace(".", "")
rutes["poliza"] = rutes["poliza"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["poliza"] = rutes["poliza"].str.strip() """


rutes.head()

In [ ]:
# derty = rutes.to_numpy().transpose().tolist()


Pacientes_SF2 = Pacientes_SF[["Rut", "Id Account Paciente"]]

In [ ]:
rutesi = pd.merge(
    left=rutes,
    right=Pacientes_SF2,
    how="left",
    left_on="rut",
    right_on="Rut",
)
rutesi.shape
rutesi.to_excel("F:/Empresas SF/reportes/rutsyempresa.xlsx", index=False)

In [ ]:
rutesi.head()

In [ ]:
rutesi_antg = pd.merge(
    left=rutesi,
    right=empresas_unidas,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)

In [ ]:
rutesi_antg.head()

In [ ]:
# rutesi_antg .to_excel("F:/Empresas SF/reportes/ruts_empresas_antiguos.xlsx", index=False)

In [ ]:
import json
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf

simpler_sf.simple_salesforce()

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [ ]:
""" query = "select Id, Account.Name, RutContact__c from AccountContactRelation "

df1 = sf.smart_query(query) """

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
# rutes = Subir_pacientes["rut"]

In [ ]:
""" df1.head() """

In [ ]:
""" df1["rut"] = df1["RutContact__c"].astype(str)
df1["rut"] = df1["rut"].str.replace(".", "")
df1["rut"] = df1["rut"].str.replace(
    "-", ""
)  # aca se trabaja con el nuevo rut desde rut1 ojooooooo
df1["rut"] = df1["rut"].str.upper()
df1["rut"] = df1["rut"].str.strip()
df1.head() """

In [ ]:
""" rutes.head() """

In [ ]:
""" rutesi = pd.merge(
    left=rutes,
    right=df1,
    how="left",
    left_on="rut",
    right_on="rut",
) """

In [ ]:
""" rutesi.to_excel("F:/Empresas SF/reportes/rutesnuevos_accountcontactrelation.xlsx", index=False) """

In [ ]:
""" df2["P_liza__c"] = df2["P_liza__c"].astype(str)
df2["P_liza__c"] = df2["P_liza__c"].str.replace(".", "")
df2["P_liza__c"] = df2["P_liza__c"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
df2["P_liza__c"] = df2["P_liza__c"].str.upper()
df2["P_liza__c"] = df2["P_liza__c"].str.strip() """

In [ ]:
modelo_nuevo = pd.merge(
    left=rutesi,
    right=df2,
    how="left",
    left_on="poliza",
    right_on="P_liza__c",
)
modelo_nuevo.to_excel("F:/Empresas SF/reportes/modelo_nuevo.xlsx", index=False)

In [ ]:
modelo = pd.DataFrame()
modelo = modelo_nuevo[["rutT", "Id Account Paciente", "Id"]]

In [ ]:
modelo.head()